# Intraday Strategy using GARCH Model 
1. Using Simulated daily data and intraday 5-min data 
2. Load Daily and 5 min data 
3. Define function to fit GARCH model on the daily data and predict 1-day ahead volatility in a rolling window 
4. Calculate prediction premium and form a daily signal from it 
5. Merge with intraday data and calculate intraday indicators to form the intraday signal 
6. Generate the position entry and hold until the end of the day 
7. Calculate final strategy returns 

In [4]:
import matplotlib.pyplot as plt 
from arch import arch_model
import pandas as pd 
import numpy as np 
import os 
import datetime as dt
import yfinance as yf  



In [20]:
# download data of these stocks 
end= dt.datetime.now()
end_date = pd.to_datetime(end)
# start = dt.datetime(2023,10,10)
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*1)

end_date

Timestamp('2024-01-20 08:09:24.862192')

In [21]:
# download using yfinance
TSLA_daily_df = yf.download(tickers='TSLA',
                 start= start_date,
                 end= end_date)

TSLA_daily_df 

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-20,128.679993,133.509995,127.349998,133.419998,133.419998,138858100
2023-01-23,135.869995,145.380005,134.270004,143.750000,143.750000,203119200
2023-01-24,143.000000,146.500000,141.100006,143.889999,143.889999,158699100
2023-01-25,141.910004,146.410004,138.070007,144.429993,144.429993,192734300
2023-01-26,159.970001,161.419998,154.759995,160.270004,160.270004,234815100
...,...,...,...,...,...,...
2024-01-12,220.080002,225.339996,217.149994,218.889999,218.889999,122889000
2024-01-16,215.100006,223.490005,212.179993,219.910004,219.910004,115355000
2024-01-17,214.860001,215.669998,212.009995,215.550003,215.550003,103164400


In [13]:
# Save the file to csv and json.
TSLA_daily_df.to_csv('/Users/shrikanthnallapati/Projects/stock_app/Data/TSLA_daily.csv')

In [22]:
# download using yfinance
TSLA_5m_df = yf.download(tickers='TSLA', period="60d", interval="5m")

TSLA_5m_df 

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-10-24 09:30:00-04:00,216.500000,218.300003,214.821396,218.050095,218.050095,7528286
2023-10-24 09:35:00-04:00,218.089905,218.800003,216.960007,218.397400,218.397400,3938770
2023-10-24 09:40:00-04:00,218.369995,219.360001,217.940002,219.250107,219.250107,3473846
2023-10-24 09:45:00-04:00,219.169998,219.699997,218.270004,218.455002,218.455002,3248788
2023-10-24 09:50:00-04:00,218.460007,219.300003,217.940002,218.803406,218.803406,2624942
...,...,...,...,...,...,...
2024-01-19 15:35:00-05:00,211.460007,211.499893,210.949997,211.020004,211.020004,754959
2024-01-19 15:40:00-05:00,211.010101,211.169998,210.820007,211.079803,211.079803,899243
2024-01-19 15:45:00-05:00,211.059998,211.630005,211.009995,211.610001,211.610001,1004741


In [25]:
# download using yfinance
TSLA_5m_df2 = yf.download(tickers='TSLA', start = '2023-10-24', period="50d", interval="15m")
TSLA_5m_df2

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['TSLA']: Exception('%ticker%: 15m data not available for startTime=1698120000 and endTime=1705757250. The requested range must be within the last 60 days.')


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
# Save the file to csv and json.
TSLA_daily_df.to_csv('/Users/shrikanthnallapati/Projects/stock_app/Data/TSLA_daily.csv')

In [ ]:
data_folder = '/Users/shrikanthnallapati/Projects/stock_app/Data/'
daily_df = pd.read_csv(os.path.join(data_folder, '.csv'))
2014-09-17 - day
2021-09-29 - 5 min
